# Reproducing Figure 1: Multiscale Entropy Analysis on Synthetic Networks

This notebook reproduces Figure 1 from our paper, which demonstrates how normalized entropy evolves across different reduction levels for four families of synthetic graphs: Ring, Barabási-Albert, Random Regular, and Grid networks.
### Overview

We implement the multiscale entropy framework described in Section 4.1 of the paper. The experiment involves:

1. Graph Generation: Creating 10 instances each of Ring, Barabási-Albert, Random Regular, and Grid graphs at three different sizes (500, 1500, and 2500 nodes)
2. Spectral Reduction: Applying graph coarsening at multiple scales (100%, 80%, 60%, 40%, 20% of original size)
3. Entropy Calculation: Computing compression-based entropy using arithmetic encoding at each reduction level
4. Normalization: Normalizing against Erdős-Rényi random graphs with matched size and density

## Imports and Setup

In [ ]:
pip install pandas==1.5.3

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/tmp/ipykernel_1441/912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
import os
import sys

##get current file directory
current_dir = os.path.dirname(os.path.abspath('__file__'))
##get the parent directory
parent_dir = os.path.dirname(current_dir)
# Add the src directory to sys.path
src_dir = os.path.join(parent_dir, '../')
sys.path.append(src_dir)

from algorithm.calculo_entropia import *
import algorithm.calculo_entropia

from algorithm.coarsening_utils import *
import algorithm.graph_utils
import algorithm.coarsening_utils as cu
from algorithm.coarsening_utils import plot_coarsening_vertical

import numpy as np
import scipy as sp

import matplotlib
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import networkx as nx
import pygsp as gsp
from pygsp import graphs
gsp.plotting.BACKEND = 'matplotlib'

import pickle

def save_graphs(graph_dict, filename):
    """
    Save the dictionary of graphs to a file.
    """
    # Convert PyGSP graphs to NetworkX graphs for easier serialization
    nx_graph_dict = {
        size: [nx.from_scipy_sparse_array(g.W) for g in graphs]
        for size, graphs in graph_dict.items()
    }
    
    with open(filename, 'wb') as f:
        pickle.dump(nx_graph_dict, f)
    print(f"Graphs saved to {filename}")

def load_graphs(filename):
    """
    Load the dictionary of graphs from a file.
    """
    with open(filename, 'rb') as f:
        nx_graph_dict = pickle.load(f)
    print(f"Graphs loaded from {filename}")
    return nx_graph_dict

In [ ]:
import pandas as pd
print(pd.__version__)

1.5.3


In [ ]:
import pickle  
# load the data 
infile = open('./CommunityFitNet_updated.pickle','rb')  
df = pickle.load(infile) 

In [ ]:
# read edge lists for all networks
df_edgelists = df['edges_id'] # column 'edges_id' in dataframe df includes the edge list 
                              # for each network 
 
# extract the edge list for the first network 
edges_orig = df_edgelists.iloc[0] # a numpy array of edge list for original graph 


## Step 1: Graph Generation
We generate synthetic networks across four families and three size scales. Each family exhibits distinct structural properties:

- Ring graphs: Regular lattice with random shortcuts (small-world-like)
- Barabási-Albert: Scale-free networks with power-law degree distribution
- Random Regular: Uniform degree distribution with random connectivity
- Grid graphs: 2D lattice structure

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pygsp import graphs
import networkx as nx
import pickle

# Setting plot functions for family

import matplotlib.pyplot as plt
import json
import numpy as np
from cycler import cycler

def get_nodes_in_range(graph_data, min_nodes, max_nodes):
    try:
        nodes = graph_data['reductions']['100']['number_nodes']
        return min_nodes <= nodes < max_nodes
    except (KeyError, TypeError):
        return False

def safe_get_entropy_value(graph_data, level):
    """Safely get entropy value, return None if not available."""
    try:
        return graph_data['reductions'][str(level)]['entropy_arithmetic']['normalized']
    except (KeyError, TypeError):
        return None

def get_valid_entropy_values(graph_data, reduction_levels):
    """Get entropy values and their corresponding levels."""
    values = []
    valid_levels = []
    for level in reduction_levels:
        value = safe_get_entropy_value(graph_data, level)
        if value is not None:
            values.append(value)
            valid_levels.append(level)
    return valid_levels, values

def get_y_axis_limits(entropy_values_list, padding=0.1):
    """Calculate appropriate y-axis limits based on data"""
    all_values = []
    
    # Handle both list of lists and list of floats
    for values in entropy_values_list:
        if isinstance(values, (list, tuple)):
            all_values.extend([v for v in values if v is not None])
        elif isinstance(values, (int, float)):
            all_values.append(values)
    
    if not all_values:
        return 0.5, 1.1
    
    min_val = min(all_values)
    max_val = max(all_values)
    
    # Add padding
    range_val = max_val - min_val
    min_val = max(0, min_val - range_val * padding)
    max_val = max_val + range_val * padding
    
    return min_val, max_val

def plot_family_specific(json_data, family_name, node_ranges, figsize=(25, 20)):
    n_ranges = len(node_ranges)
    n_cols = 2
    n_rows = (n_ranges + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    if n_rows == 1 and n_cols == 1:
        axes = np.array([axes])
    elif n_rows == 1:
        axes = np.array([axes])
    elif n_cols == 1:
        axes = np.array([[ax] for ax in axes])
    axes = axes.flatten()
    
    reduction_levels = [100, 80, 60, 40, 20]
    family_data = json_data[family_name]
    
    # First pass to get global y-axis limits
    all_entropy_values = []
    for min_nodes, max_nodes in node_ranges:
        filtered_graphs = {name: data for name, data in family_data.items() 
                         if get_nodes_in_range(data, min_nodes, max_nodes)}
        
        for graph_data in filtered_graphs.values():
            _, values = get_valid_entropy_values(graph_data, reduction_levels)
            all_entropy_values.extend(values)  # values is already a list
    
    y_min, y_max = get_y_axis_limits(all_entropy_values)
    
    # Plot each range in its own subplot
    for range_idx, (min_nodes, max_nodes) in enumerate(node_ranges):
        ax = axes[range_idx]
        range_label = f"{min_nodes}-{max_nodes if max_nodes != float('inf') else '+'}"
        
        filtered_graphs = {name: data for name, data in family_data.items() 
                         if get_nodes_in_range(data, min_nodes, max_nodes)}
        
        if filtered_graphs:
            n_graphs = len(filtered_graphs)
            colors = plt.cm.rainbow(np.linspace(0, 1, n_graphs))
            
            for graph_idx, (graph_name, graph_data) in enumerate(filtered_graphs.items()):
                valid_levels, entropy_values = get_valid_entropy_values(graph_data, reduction_levels)
                
                if valid_levels:
                    node_count = graph_data['reductions']['100']['number_nodes']
                    short_name = f"{graph_name[:20]}... ({node_count})"
                    
                    ax.plot(valid_levels, entropy_values,
                           marker='o',
                           linestyle='-',
                           linewidth=1.5,
                           markersize=6,
                           label=short_name,
                           color=colors[graph_idx],
                           alpha=0.7)
        
        ax.set_xlabel('Graph Portion (%)', fontsize=10)
        ax.set_ylabel('Normalized Entropy', fontsize=10)
        ax.set_title(f'Range: {range_label} nodes (n={len(filtered_graphs)})', 
                    fontsize=12)
        
        ax.grid(True, linestyle='--', alpha=0.3)
        ax.set_xticks(reduction_levels)
        ax.tick_params(axis='both', which='major', labelsize=8)
        ax.invert_xaxis()
        ax.set_ylim(y_min, y_max)
        
        if len(filtered_graphs) > 0:
            ax.legend(bbox_to_anchor=(1.02, 1),
                     loc='upper left',
                     borderaxespad=0.,
                     fontsize=8,
                     ncol=1)
    
    # Remove empty subplots
    for idx in range(len(node_ranges), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.suptitle(f'Entropy Reduction Patterns for {family_name} Networks',
                 fontsize=16,
                 fontweight='bold',
                 y=0.95)
    
    plt.tight_layout(rect=[0, 0, 0.95, 0.95])
    return fig

# Set parameters
node_sizes = [2500, 1500, 500]
num_graphs = 10

# Dictionary to store all graphs
# Structure: graphs_dict[graph_type][node_size] = list_of_graphs
graphs_dict = {
    'ring': {size: [] for size in node_sizes},
    'barabasi': {size: [] for size in node_sizes},
    'regular': {size: [] for size in node_sizes},
    'grid': {size: [] for size in node_sizes}
}

# Create graphs for each type and size
for N in node_sizes:
    print(f"\nCreating graphs with {N} nodes...")
    
    for i in range(num_graphs):
        print(f"Creating set {i+1}/{num_graphs}...")
        
        # 1. Ring Graph with random edges
        G_ring = graphs.Ring(N=N, k=1)
        p = 0.005
        for u in range(N):
            for v in range(u+2, N):
                if np.random.random() < p:
                    G_ring.W[u, v] = G_ring.W[v, u] = 1
        graphs_dict['ring'][N].append(G_ring)
        
        # 2. Barabasi-Albert Graph
        G_ba = graphs.BarabasiAlbert(N=N)
        graphs_dict['barabasi'][N].append(G_ba)
        
        # 3. Random Regular Graph
        G_nx_regular = nx.random_regular_graph(d=5, n=N)
        W_regular = nx.adjacency_matrix(G_nx_regular)
        G_regular = graphs.Graph(W_regular)
        graphs_dict['regular'][N].append(G_regular)
        
        # 4. Grid Graph
        # Calculate grid dimensions
        m, n = int(np.sqrt(N)), int(np.sqrt(N))
        while m * n != N:
            m -= 1
            n = N // m
        G_nx_grid = nx.grid_2d_graph(m=m, n=n)
        W_grid = nx.adjacency_matrix(G_nx_grid)
        G_grid = graphs.Graph(W_grid)
        graphs_dict['grid'][N].append(G_grid)

# Save all graphs in one file
print("\nSaving all graphs...")
with open('all_graphs.pkl', 'wb') as f:
    pickle.dump(graphs_dict, f)
print("Saved all graphs to all_graphs.pkl")

# Print information about the saved graphs
print("\nGraph Information:")
for graph_type in graphs_dict:
    print(f"\n{graph_type.upper()} Graphs:")
    for size in node_sizes:
        num_graphs = len(graphs_dict[graph_type][size])
        sample_graph = graphs_dict[graph_type][size][0]
        print(f"\nSize {size}:")
        print(f"Number of graphs: {num_graphs}")
        print(f"Nodes per graph: {sample_graph.N}")
        print(f"Edges per graph: {sample_graph.W.nnz//2}")

In [2]:
print(graphs_dict)

{'ring': {2500: [<pygsp.graphs.ring.Ring object at 0x7f2f25309520>, <pygsp.graphs.ring.Ring object at 0x7f2ee40ee070>, <pygsp.graphs.ring.Ring object at 0x7f2ee2f08e20>, <pygsp.graphs.ring.Ring object at 0x7f2ee41d8d30>, <pygsp.graphs.ring.Ring object at 0x7f2ee30bbdf0>, <pygsp.graphs.ring.Ring object at 0x7f2ee1e6fd30>, <pygsp.graphs.ring.Ring object at 0x7f2ee38a9dc0>, <pygsp.graphs.ring.Ring object at 0x7f2ee0e11d30>, <pygsp.graphs.ring.Ring object at 0x7f2ee2fd0d60>, <pygsp.graphs.ring.Ring object at 0x7f2ee2067fa0>], 1500: [<pygsp.graphs.ring.Ring object at 0x7f2ee388b490>, <pygsp.graphs.ring.Ring object at 0x7f2edf8c5f40>, <pygsp.graphs.ring.Ring object at 0x7f2ee1043fa0>, <pygsp.graphs.ring.Ring object at 0x7f2ee0514e80>, <pygsp.graphs.ring.Ring object at 0x7f2edf4eaf70>, <pygsp.graphs.ring.Ring object at 0x7f2ee012c580>, <pygsp.graphs.ring.Ring object at 0x7f2edecbdd30>, <pygsp.graphs.ring.Ring object at 0x7f2edf79a0a0>, <pygsp.graphs.ring.Ring object at 0x7f2edea79cd0>, <pygsp

## Step 2: Multiscale Reduction and Entropy Calculation
In this step, we apply the spectral coarsening algorithm (Section 3.1) to each graph at multiple reduction levels, then compute the normalized compression entropy at each scale. This process generates the data file synthetic_graphs_analysis.json containing entropy measurements across all graphs and reduction levels.
Note: The code for this step is implemented in the project files coarsening_utils.py and calculo_entropia.py. Run the reduction pipeline to generate the analysis file before proceeding to visualization.

## Step 3: Visualization - Creating Figure 1
Now we load the processed data and generate the visualization that appears as Figure 1 in the paper. We create two types of plots:

1. By network size: Comparing all four families at each size scale
2. By network family: Showing how each family behaves across different sizes

In [ ]:
import networkx as nx
import numpy as np
from pygsp import graphs
import json

def get_graph_edges(graph):
    """Safely get number of edges from a PyGSP graph"""
    try:
        # Try to use the stored number of edges
        return graph.Ne
    except AttributeError:
        try:
            # Calculate from the weight matrix
            return int(graph.W.nnz / 2)  # Divide by 2 for undirected graphs
        except:
            # Count non-zero elements in adjacency matrix
            return int(np.sum(graph.W.toarray() > 0) / 2)

def process_synthetic_graphs(synthetic_graphs):
    """Convert synthetic graphs data into our analysis format"""
    json_data = {}
    
    for family_name, size_dict in synthetic_graphs.items():
        print(f"\nProcessing {family_name} graphs...")
        family_data = {}
        
        for size, graph_list in size_dict.items():
            print(f"Processing size {size}, {len(graph_list)} graphs")
            
            # Process each graph in the list
            for idx, graph in enumerate(graph_list):
                graph_name = f"{family_name}_{size}_{idx}"
                print(f"Processing {graph_name}")
                
                # Initialize reduction data
                reductions = {}
                current_graph = graph
                
                try:
                    # Calculate initial metrics
                    N = current_graph.N  # Original number of nodes
                    E = get_graph_edges(current_graph)  # Original number of edges
                    
                    # Store original graph metrics (100%)
                    reductions['100'] = {
                        'number_nodes': N,
                        'number_edges': E,
                        'ave_degree': 2 * E / N if N > 0 else 0,
                    }
                    
                    # Perform reductions
                    for reduction in [80, 60, 40, 20]:
                        try:
                            # Calculate reduction ratio
                            r = 1 - (reduction/100)
                            
                            # Perform coarsening
                            C, Gc, Call, Gall = coarsen(current_graph, K=10, r=r)
                            
                            if Gc is not None:
                                n = Gc.N  # Number of nodes after reduction
                                e = get_graph_edges(Gc)  # Number of edges after reduction
                                
                                reductions[str(reduction)] = {
                                    'number_nodes': n,
                                    'number_edges': e,
                                    'ave_degree': 2 * e / n if n > 0 else 0,
                                }
                                
                                # Calculate entropy metrics
                                G_nx = nx.from_scipy_sparse_array(Gc.W)
                                entropy_metrics = get_entropy_metadata_aritmethicEncoding(G_nx)
                                
                                reductions[str(reduction)]['entropy_arithmetic'] = {
                                    'graph': entropy_metrics['Grafo'],
                                    'random': entropy_metrics['Grafo_r'],
                                    'normalized': entropy_metrics['Entropy Normalizado']
                                }
                                
                        except Exception as e:
                            print(f"Error in reduction {reduction}% for {graph_name}: {str(e)}")
                            continue
                    
                    # Calculate entropy metrics for original graph
                    try:
                        G_nx = nx.from_scipy_sparse_array(graph.W)
                        entropy_metrics = get_entropy_metadata_aritmethicEncoding(G_nx)
                        reductions['100']['entropy_arithmetic'] = {
                            'graph': entropy_metrics['Grafo'],
                            'random': entropy_metrics['Grafo_r'],
                            'normalized': entropy_metrics['Entropy Normalizado']
                        }
                    except Exception as e:
                        print(f"Error calculating original entropy for {graph_name}: {str(e)}")
                    
                    # Add to family data
                    family_data[graph_name] = {
                        'Name': graph_name,
                        'Subdomain': 'Synthetic',
                        'Node_Type': 'Vertex',
                        'Edge_Type': family_name,
                        'reductions': reductions
                    }
                    
                except Exception as e:
                    print(f"Error processing graph {graph_name}: {str(e)}")
                    continue
        
        json_data[family_name] = family_data
    
    return json_data

# Define ranges for synthetic graphs
synthetic_ranges = {
    'ring': [(0, 1000), (1000, 2000), (2000, float('inf'))],
    'barabasi': [(0, 1000), (1000, 2000), (2000, float('inf'))],
    'regular': [(0, 1000), (1000, 2000), (2000, float('inf'))],
    'grid': [(0, 1000), (1000, 2000), (2000, float('inf'))]
}

# Load and process the data
try:
    print("Loading synthetic graphs...")
    with open('all_graphs.pkl', 'rb') as f:
        synthetic_graphs = pickle.load(f)
    
    print("Processing graphs...")
    json_data = process_synthetic_graphs(synthetic_graphs)
    
    print("Saving processed data...")
    with open('synthetic_graphs_analysis.json', 'w') as f:
        json.dump(json_data, f, indent=2)
    
    print("Creating plots...")
    for family_name, ranges in synthetic_ranges.items():
        try:
            fig = plot_family_specific(json_data, family_name, ranges)
            if fig is not None:
                plt.savefig(f'entropy_{family_name}_synthetic_by_ranges.png',
                           bbox_inches='tight',
                           dpi=300)
                plt.close()
        except Exception as e:
            print(f"Error plotting {family_name}: {str(e)}")
            
except Exception as e:
    print(f"Error in main execution: {str(e)}")
    import traceback
    traceback.print_exc()

In [10]:
def plot_size_specific(json_data, node_size, figsize=(15, 10)):
    """Plot all families for a specific node size."""
    fig, ax = plt.subplots(figsize=figsize)
    reduction_levels = [100, 80, 60, 40, 20]
    
    # Different color for each family
    colors = {'ring': 'blue', 'barabasi': 'red', 'regular': 'green', 'grid': 'purple'}
    markers = {'ring': 'o', 'barabasi': 's', 'regular': '^', 'grid': 'D'}
    
    for family_name in ['ring', 'barabasi', 'regular', 'grid']:
        entropy_values = []
        
        # Get all graphs of this size for this family
        for graph_name, graph_data in json_data[family_name].items():
            if f"{node_size}" in graph_name:
                values = [graph_data['reductions'][str(level)]['entropy_arithmetic']['normalized'] 
                         for level in reduction_levels]
                entropy_values.append(values)
        
        # Plot average with error bars
        if entropy_values:
            mean_values = np.mean(entropy_values, axis=0)
            std_values = np.std(entropy_values, axis=0)
            
            ax.errorbar(reduction_levels, mean_values, yerr=std_values,
                       label=f'{family_name}',
                       color=colors[family_name],
                       marker=markers[family_name],
                       markersize=8,
                       linewidth=2,
                       capsize=5)
    
    ax.set_xlabel('Graph Portion (%)', fontsize=12)
    ax.set_ylabel('Normalized Entropy', fontsize=12)
    ax.set_title(f'Entropy Reduction Patterns for {node_size}-node Networks', 
                 fontsize=14)
    ax.grid(True, linestyle='--', alpha=0.3)
    ax.set_xticks(reduction_levels)
    ax.invert_xaxis()
    ax.legend()
    plt.tight_layout()
    return fig

def plot_family_sizes(json_data, family_name, figsize=(15, 10)):
    """Plot all sizes for a specific family."""
    fig, ax = plt.subplots(figsize=figsize)
    reduction_levels = [100, 80, 60, 40, 20]
    
    colors = {500: 'blue', 1500: 'red', 2500: 'green'}
    markers = {500: 'o', 1500: 's', 2500: '^'}
    
    for size in [500, 1500, 2500]:
        entropy_values = []
        
        # Get all graphs of this size
        for graph_name, graph_data in json_data[family_name].items():
            if f"{size}" in graph_name:
                values = [graph_data['reductions'][str(level)]['entropy_arithmetic']['normalized'] 
                         for level in reduction_levels]
                entropy_values.append(values)
        
        # Plot average with error bars
        if entropy_values:
            mean_values = np.mean(entropy_values, axis=0)
            std_values = np.std(entropy_values, axis=0)
            
            ax.errorbar(reduction_levels, mean_values, yerr=std_values,
                       label=f'{size} nodes',
                       color=colors[size],
                       marker=markers[size],
                       markersize=8,
                       linewidth=2,
                       capsize=5)
    
    ax.set_xlabel('Graph Portion (%)', fontsize=12)
    ax.set_ylabel('Normalized Entropy', fontsize=12)
    ax.set_title(f'Entropy Reduction Patterns for {family_name} Networks', 
                 fontsize=14)
    ax.grid(True, linestyle='--', alpha=0.3)
    ax.set_xticks(reduction_levels)
    ax.invert_xaxis()
    ax.legend()
    plt.tight_layout()
    return fig

# Load data and create plots
with open('synthetic_graphs_analysis.json', 'r') as f:
    json_data = json.load(f)

# Plot by size
for size in [2500, 1500, 500]:
    fig = plot_size_specific(json_data, size)
    plt.savefig(f'entropy_size_{size}.png', bbox_inches='tight', dpi=300)
    plt.close()

# Plot by family
for family in ['ring', 'barabasi', 'regular', 'grid']:
    fig = plot_family_sizes(json_data, family)
    plt.savefig(f'entropy_family_{family}.png', bbox_inches='tight', dpi=300)
    plt.close()